In [1]:
import os
import sys
import getpass
from binance.spot import Spot

__file__ = %pwd
loc_list = os.path.abspath(__file__).split(os.sep)
HOME_LOC = os.path.join(os.sep, *loc_list[:-2])
sys.path.append(HOME_LOC)
os.chdir(HOME_LOC)
from CODE.Utils.encrypt import *
from CODE.Utils.encrypt import Encrypted_API_key
from CODE.Utils.encrypt import Encrypted_API_secret
from CODE.Utils.utils import *
from CODE.Utils.indicators import *
from CODE.Utils.normalize import *
from CODE.Runner import *

/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3
/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3


In [2]:
file_paths = load_and_sort_files(os.path.join(HOME_LOC, "DATA", "RAW", "1d"))[:50]
train_X, train_Y, test_X, test_Y = split_data(
    file_paths, train_ratio=0.85, x_length=58, y_length=2
)

Processing Training Files:   0%|          | 0/42 [00:00<?, ?it/s]

Processing Testing Files: 100%|██████████| 8/8 [00:00<00:00, 11.84it/s]


In [3]:
print(train_X.shape, train_Y.shape)

(42, 47, 23) (42, 2)


In [4]:
train_Y

array([[-0.0721127 ,  0.07770241],
       [-0.0721127 ,  0.07770241],
       [ 0.02862476,  0.1328362 ],
       [ 0.02862476,  0.1328362 ],
       [ 0.02862476,  0.1328362 ],
       [ 0.1328362 ,  0.09207511],
       [ 0.09207511,  0.00639166],
       [ 0.09207511,  0.00639166],
       [ 0.09207511,  0.00639166],
       [ 0.00639166,  0.02053191],
       [ 0.02053191,  0.04671114],
       [ 0.11849299,  0.00898414],
       [ 0.00898414,  0.13927178],
       [ 0.13927178, -0.03408211],
       [-0.03408211, -0.08666399],
       [-0.08666399,  0.10091928],
       [ 0.10091928,  0.10698876],
       [-0.02955289, -0.05642168],
       [-0.05642168, -0.21321794],
       [-0.05642168, -0.21321794],
       [-0.05642168, -0.21321794],
       [-0.21321794,  0.        ],
       [-0.21321794,  0.        ],
       [-0.21321794,  0.        ],
       [ 0.        , -0.006     ],
       [ 0.        , -0.006     ],
       [ 0.        , -0.006     ],
       [-0.006     ,  0.03873239],
       [-0.006     ,

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Simple1DCNN(nn.Module):
    def __init__(self, num_channels, output_size, data_len):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(num_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(
            128 * (data_len // 2 // 2), 128
        )  # Adjust the input features for the linear layer
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        # Reshape x to (batch_size, channels, length)
        x = x.view(x.size(0), 23, -1)  # assuming input x has shape (batch_size, 47, 23)
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor for the linear layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(42, 47, 23) (42, 2) (8, 47, 23) (8, 2)


In [7]:
# 创建 Simple1DCNN 模型实例
num_channels = train_X.shape[-1]  # 通道数，对应于特征数
output_size = train_Y.shape[-1]  # 输出尺寸
batch_size = 15  # 批处理大小
model_p = {
    "num_channels": num_channels,
    "output_size": output_size,
    "data_len": train_X.shape[1],
}  # 保存模型参数，用于初始化 GroupedTrainer
# 创建 GroupedTrainer 实例
trainer = GroupedTrainer(
    model=Simple1DCNN,  # 使用刚刚创建的模型
    data=(train_X, test_X, train_Y, test_Y),  # 传入训练和测试数据
    batch_size=batch_size,  # 指定批处理大小
    model_p=model_p,  # model_p为空，因为模型已经被初始化
    learning_rate=0.01,  # 学习率
    weight_decay=0.0,  # 权重衰减（正则化参数）
)

# 设置损失函数为均方误差（适用于回归任务）
trainer.criterion = nn.MSELoss()

# 打印模型结构（可选）
print(trainer.model)

Simple1DCNN(
  (conv1): Conv1d(23, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=1408, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [28]:
trainer.train(5000, iter_n=100)

Sharpe Ratio: 0.85, Max Drawdown: 0.10, Annualized Return: 1150.67


0.02919111485243775

In [9]:
trainer.train_loader.dataset[0][0].shape

torch.Size([47, 23])

In [10]:
trainer.optimizer.zero_grad()
trainer.model(trainer.train_loader.dataset[1][0].to(trainer.device))

RuntimeError: max_pool1d() Invalid computed output size: 0